In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

test_filename = 'test.csv'
train_filename = 'train.csv'
test_df = pd.read_csv(test_filename)
train_df = pd.read_csv(train_filename)



In [2]:
def pos_pixel(size, i ,j):
    return i*size + j

def symmetry(image):
  size = 28
  sv = 0
  sh = 0
  for i in range(size):
      for j in range(int(size/2)):
        
          p1 = image[pos_pixel(size, i, j)]
          p2 = image[pos_pixel(size, i, (size-1)-j)]
          sv += abs(p1 - p2)
                
          p1 = image[pos_pixel(size, j, i)]
          p2 = image[pos_pixel(size, (size-1)-j, i)]
          sh += abs(p1 - p2)

  sv = sv/255
  sh = sh/255
        
  #s = sv
  #s = sh
  #s = pow(sv + sh, 2)
  s = sv + sh
  #s = abs(sv - sh)

        
  return (s)
  
def intensity(image):
  return (np.sum(image)/255.0)

In [3]:
new_train_df = []
new_test_df = []

for row in train_df.iterrows():
    data_row = row[1]

    id = data_row[0]
    label = data_row[1]
    imagem = data_row[2:]

    intensity_ = intensity(imagem)
    symmetry_ = symmetry(imagem)
    new_train_df.append([label,intensity_,symmetry_])

for row in test_df.iterrows():
    data_row = row[1]

    id = data_row[0]
    label = data_row[1]
    imagem = data_row[2:]

    intensity_ = intensity(imagem)
    symmetry_ = symmetry(imagem)
    new_test_df.append([label,intensity_,symmetry_])

In [4]:
new_train_df = pd.DataFrame(new_train_df, columns=['label', 'intensity', 'symmetry'])
new_test_df = pd.DataFrame(new_test_df, columns=['label', 'intensity', 'symmetry'])

In [5]:
new_test_df.to_csv('test_redu.csv', index= False)
new_train_df.to_csv('train_redu.csv', index= False)